## JOIN

In [2]:
# Import Libraries
#초기 설정및 시스템 라이브러리
import platform
import warnings

# 데이터 시각화 라이브러리
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
print(platform.system())
warnings.filterwarnings('ignore')

# 행,열,결과값 생략 없이 보기,세팅
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
%matplotlib inline

# 시각화 OS별 한글폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 폰트 설정
elif platform.system() == 'Mac':
    plt.rcParams['font.family'] = 'AppleGothic'  # Mac 폰트 설정
    
print("="*60)
print("라이브러리 로드 완료!")
print("한글 폰트 설정 완료!")
print("="*60)

Windows
라이브러리 로드 완료!
한글 폰트 설정 완료!


In [3]:
# Load Dataset
obj   = pd.read_csv("./data/clean/clean_objects_final.csv")         # objects
frs   = pd.read_csv("./data/clean/clean_fr_final.csv")              # funding_rounds
inv   = pd.read_csv("./data/clean/clean_investments_final.csv")     # investments
acq   = pd.read_csv("./data/clean/clean_acquisitions_final.csv")    # acquisitions
ipo   = pd.read_csv("./data/clean/clean_ipos_final.csv")            # ipos
rel   = pd.read_csv("./data/clean/clean_relationships_final.csv")   # relationships
off   = pd.read_csv("./data/clean/clean_offices_final.csv")         # offices
print("="*60)
print("데이터셋 로드 완료!")
print("="*60)

데이터셋 로드 완료!


In [4]:
print("obj: ", obj.shape)
print("frs: ", frs.shape)
print("inv: ", inv.shape)
print("acq: ", acq.shape)
print("ipo: ", ipo.shape)
print("rel: ", rel.shape)
print("off: ", off.shape)

obj:  (462620, 45)
frs:  (52928, 17)
inv:  (80902, 4)
acq:  (9562, 11)
ipo:  (1259, 16)
rel:  (402412, 10)
off:  (112718, 11)


### START-UP

#### 투자 성공률 (핵심 테이블: FRS,INV, ACQ, IPO, OFF)
<span style="font-size: 15px;">
grain: 스타트업 id x 투자라운드 id
</span>

In [5]:
cols_use = [
    #objects
    "objects_cfpr_id"       , "founded_at"          , "closed_at"           ,
    "description"           , "country_code"        , "obj_city_fixed"      , 
    "first_investment_at"   , "last_investment_at"  , "investment_rounds"   ,
    "invested_companies"    , "first_funding_at"    , "last_funding_at"     ,
    "funding_rounds"        , "funding_total_usd"   , "relationships"       ,
    "cat_obj_status"        , "obj_category_filled" , "cat_obj_overview"    ,
    "obj_state_filled"      , "is_obj_funding_total_usd_private"  
]
obj = obj[cols_use].copy()
print(obj.shape)
print(obj.columns)

(462620, 20)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private'],
      dtype='object')


##### ◼ 파생변수 생성

<span style="font-size: 15px;">
<code>success_flag</code> : 성공적인 EXIT을 달성 플래그
</span>

In [6]:
cond1 = obj["objects_cfpr_id"].astype(str).str.startswith("c:")     # 회사 조건
cond2 = obj["objects_cfpr_id"].isin(ipo["ipos_c_id"].dropna())      # IPO 성공 조건
cond3 = obj["objects_cfpr_id"].isin(acq["acquired_c_id"].dropna())  # 인수(M&A) 성공 조건

In [7]:
# cond4: funding_rounds의 num_fr_type 조건
frs["funded_at"] = pd.to_datetime(frs["funded_at"], errors="coerce") # 날짜 변환 

# 회사별 num_fr_type 범위 
fr_type_range = (
    frs.dropna(subset=["fr_c_id", "num_fr_type"])
       .groupby("fr_c_id")["num_fr_type"]
       .agg(num_fr_type_min="min", num_fr_type_max="max")
)

# cond4 해석: 
# 라운드 타입이 최소 1단계라도 진행(= max - min >= 1)
# max_type < 5 
cond4_1 = (fr_type_range["num_fr_type_max"] - fr_type_range["num_fr_type_min"]) >= 1
cond4_2 = fr_type_range["num_fr_type_max"] < 5
fr_type_range["is_round_on"] = cond4_1 & cond4_2 # cond4 라운드 진행 조건
fr_type_range

,num_fr_type_min,num_fr_type_max,is_round_on
fr_c_id,,,
c:1,1,3,True
c:1001,1,1,False
c:10014,0,0,False
c:10015,1,99,False
c:100155,2,99,False
...,...,...,...
c:99853,0,0,False
c:9989,0,0,False
c:9994,0,0,False


In [8]:
# obj에 회사별 cond4 붙이기
obj_tmp = obj.merge(fr_type_range[["is_round_on"]], left_on="objects_cfpr_id", right_index=True, how="left")
obj_tmp["is_round_on"] = obj_tmp["is_round_on"].fillna(False)

# 성공 플래그 (회사면서, IPO or 인수 or 라운드)
obj["success_flag"] = (cond1 & (cond2 | cond3 | obj_tmp["is_round_on"])).astype(int)
obj

,objects_cfpr_id,founded_at,closed_at,description,country_code,obj_city_fixed,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,relationships,cat_obj_status,obj_category_filled,cat_obj_overview,obj_state_filled,is_obj_funding_total_usd_private,success_flag
0,c:1,2005-10-17,NaN,Technology Platform Company,USA,seattle,NaN,NaN,0,0,2005-10-01,2008-05-19,3.0,39750000.0,17,operating,web,Software & Technology,WA,0,1
1,c:10,NaN,NaN,NaN,USA,culver city,NaN,NaN,0,0,NaN,NaN,NaN,NaN,6,acquired,games_video,Software & Technology,CA,1,1
2,c:100,NaN,NaN,NaN,USA,san mateo,NaN,NaN,0,0,NaN,NaN,NaN,NaN,12,acquired,games_video,Software & Technology,CA,1,1
3,c:10000,2008-07-26,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0.0,0.0,0,operating,network_hosting,Software & Technology,NaN,0,0
4,c:10001,2008-07-26,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0.0,0.0,0,operating,games_video,Software & Technology,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462615,r:9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0.0,0.0,0,operating,software,Software & Technology,NaN,0,0
462616,r:9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0.0,0.0,0,operating,software,Software & Technology,NaN,0,0
462617,r:9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0.0,0.0,0,operating,software,Software & Technology,NaN,0,0
462618,r:9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0.0,0.0,0,operating,software,Software & Technology,NaN,0,0


In [9]:
obj["cat_obj_status"].value_counts()

cat_obj_status
operating    449319
acquired       9394
closed         2773
ipo            1134
Name: count, dtype: int64

In [10]:
# closed상태는 success_flag 0
cond5 = (obj["cat_obj_status"] ==  "closed") 
obj.loc[cond5, "success_flag"] = 0 

obj[cond5 & obj["success_flag"]==1].value_counts().sum() # 적용 확인

np.int64(0)

In [11]:
obj[["objects_cfpr_id", "success_flag"]].head()

,objects_cfpr_id,success_flag
0,c:1,1
1,c:10,1
2,c:100,1
3,c:10000,0
4,c:10001,0


<span style="font-size: 15px;">
<code>size_bin</code> : 기업의 규모 구간
</span>

In [12]:
# 회사 정보만 필터링
cond1 = obj["objects_cfpr_id"].astype(str).str.startswith("c:")
obj_size_bin = obj.loc[cond1].copy()
display(obj_size_bin["objects_cfpr_id"].str[0].value_counts())

# quantile 기반 bins
q = obj_size_bin["relationships"].quantile([0.25, 0.5, 0.75])
print("bin 기준\n", q)

bins = [-1, q[0.25], q[0.5], q[0.75], obj_size_bin["relationships"].max()]
labels = ["초소형팀", "소형팀", "중형팀", "대형팀"]

obj_size_bin["size_bin"] = pd.cut(
    obj_size_bin["relationships"],
    bins=bins,
    labels=labels,
    include_lowest=True  # 최소값도 포함
)

display(obj_size_bin["size_bin"].value_counts())
# 초소형 66885 | 소형 70569 | 중형 23258 | 대형 35837

# obj에 size_bin 병합 → 회사가 아닌 row는 size_bin이 NaN으로 남음
obj = obj.merge(
    obj_size_bin[["objects_cfpr_id", "size_bin"]],
    on="objects_cfpr_id",
    how="left"
)
display(obj[["objects_cfpr_id", "relationships", "size_bin"]].head())

objects_cfpr_id
c    196549
Name: count, dtype: int64

bin 기준
 0.25    0.0
0.50    1.0
0.75    2.0
Name: relationships, dtype: float64


size_bin
소형팀     70569
초소형팀    66885
대형팀     35837
중형팀     23258
Name: count, dtype: int64

,objects_cfpr_id,relationships,size_bin
0,c:1,17,대형팀
1,c:10,6,대형팀
2,c:100,12,대형팀
3,c:10000,0,초소형팀
4,c:10001,0,초소형팀


<span style="font-size: 15px;">
<code>time_to_last_round</code> : 첫 투자 → 마지막 투자 기간(개월)
</span>

In [13]:
obj["first_funding_at"] = pd.to_datetime(obj["first_funding_at"], errors="coerce")
obj["last_funding_at"]  = pd.to_datetime(obj["last_funding_at"],  errors="coerce")

# object에 있는 first_funding_at, last_funding_at 사용
obj["time_to_last_round_months"] = (
    (obj["last_funding_at"] - obj["first_funding_at"]).dt.days / 30
)
obj["time_to_last_round_months"] = obj["time_to_last_round_months"].round().astype("Int64") # 정수형으로 변경

# 유효하지 않은 값(NaT, 음수)을 NaN으로 🚨변경
cond1 = obj["first_funding_at"].notna()
cond2 = obj["last_funding_at"].notna()
cond3 = (obj["last_funding_at"] >= obj["first_funding_at"]) # 마지막 < 첫 번째
mask_valid = ( 
    cond1 & cond2 & cond3 
)

obj.loc[~mask_valid, "time_to_last_round_months"] = np.nan
display(obj.loc[mask_valid, ["first_funding_at", "last_funding_at", "time_to_last_round_months"]].head())

,first_funding_at,last_funding_at,time_to_last_round_months
0,2005-10-01,2008-05-19,32
13,2008-02-26,2008-02-26,0
18,2008-09-01,2008-09-01,0
19,2008-10-10,2013-08-13,59
20,2011-03-08,2012-01-26,11


In [14]:
# obj frs
obj_frs = obj.merge(
    frs,
    left_on ="objects_cfpr_id", 
    right_on="fr_c_id",
    how="left"
)
print(obj_frs.shape)
print(obj_frs.columns)

(483539, 40)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private'],
      dtype='object')


In [15]:
obj_frs[["objects_cfpr_id", "funding_round_id"]].duplicated().sum()

np.int64(0)

<span style="font-size: 15px;">
<code>round_tempo_months</code> : 투자 템포(개월)
</span>

In [23]:
# 회사별 라운드 간격 계산
obj_frs = (obj_frs.sort_values(['fr_c_id','funded_at']))
obj_frs['prev_round_date'] = obj_frs.groupby('fr_c_id')['funded_at'].shift(1)

obj_frs['round_tempo_days'] = (obj_frs['funded_at'] - obj_frs['prev_round_date']).dt.days
obj_frs["round_tempo_days"] = obj_frs["round_tempo_days"].astype("Int64") # 정수형으로 변경

obj_frs['round_tempo_months'] = round(obj_frs['round_tempo_days'] / 30) 
obj_frs["round_tempo_months"] = obj_frs["round_tempo_months"].astype("Int64") # 정수형으로 변경
display(obj_frs[["fr_c_id","funded_at", "round_tempo_days","round_tempo_months"]])

,fr_c_id,funded_at,round_tempo_days,round_tempo_months
0,c:1,2005-10-01,<NA>,<NA>
1,c:1,2007-01-01,457,15
2,c:1,2008-05-19,504,17
15,c:1001,2008-02-26,<NA>,<NA>
20,c:10014,2008-09-01,<NA>,<NA>
...,...,...,...,...
483534,NaN,NaT,<NA>,<NA>
483535,NaN,NaT,<NA>,<NA>
483536,NaN,NaT,<NA>,<NA>
483537,NaN,NaT,<NA>,<NA>


In [17]:
# # 회사별 median round_tempo_months 계산
# tempo_by_company = (
#     obj_frs
#     .groupby("fr_c_id")
#     .agg(    
#         round_tempo_months=("round_tempo_months", "median") 
#     )
#     .reset_index()
#     .rename(columns={"fr_c_id": "objects_cfpr_id"})
# )
# tempo_by_company["round_tempo_months"] = tempo_by_company["round_tempo_months"].astype("Int64") # 정수형으로 변경
# tempo_by_company[tempo_by_company["round_tempo_months"].notna()].head()

In [24]:
print(obj_frs.shape)
print(obj_frs.columns)

(483539, 43)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months'],
      dtype='object')


In [25]:
obj_frs[["objects_cfpr_id", "funding_round_id"]].duplicated().sum()

np.int64(0)

In [20]:
# # obj에 회사별 median round_tempo_months 병합
# obj = obj_frs.merge(
#     tempo_by_company,
#     left_on="objects_cfpr_id",
#     right_on="fr_c_id",
#     how="left"
# )

In [26]:
print(obj.shape)
print(obj.columns)

(462620, 23)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months'],
      dtype='object')


In [28]:
cond1 = (obj_frs["round_tempo_months"].notna())
obj_frs.loc[cond1, ["objects_cfpr_id", "round_tempo_months"]].head()

,objects_cfpr_id,round_tempo_months
1,c:1,15
2,c:1,17
22,c:10015,23
25,c:10015,16
23,c:10015,1


In [29]:
print(obj_frs.shape)
print(obj_frs.columns)

(483539, 43)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months'],
      dtype='object')


##### ◼ acquisitions

In [30]:
acq.columns

Index(['acquisition_id', 'acquiring_c_id', 'acquired_c_id', 'term_code',
       'price_amount', 'price_currency_code', 'acquired_at',
       'is_acq_price_private', 'is_acquisitions_acq_at_missing',
       'price_amount_usd', 'acqusition_currency_rate'],
      dtype='object')

In [31]:
cols_use = [
    #acquisitions
    "acquisition_id"    , "acquired_c_id"       , "acquiring_c_id"  , 
    "acquired_at"       , "is_acq_price_private", "price_amount_usd" 
]
acq = acq[cols_use].copy()
print(acq.columns)

Index(['acquisition_id', 'acquired_c_id', 'acquiring_c_id', 'acquired_at',
       'is_acq_price_private', 'price_amount_usd'],
      dtype='object')


In [32]:
acq["acquired_at"] = pd.to_datetime(acq["acquired_at"], errors='coerce')
acq_by_company = (
    acq
    .groupby("acquired_c_id")
    .agg(# 첫번째 이벤트
            acquisition_id        = ("acquisition_id","first"),
            acquiring_c_id        = ("acquiring_c_id","first"),
            acquired_at           = ("acquired_at","min"), 
            is_acq_price_private  = ("is_acq_price_private","max"),
            price_amount_usd      = ("price_amount_usd","first")
        )
).reset_index()
acq_by_company

,acquired_c_id,acquisition_id,acquiring_c_id,acquired_at,is_acq_price_private,price_amount_usd
0,c:10,1,c:11,2007-05-30,0,20000000.0
1,c:100,20,c:377,2005-05-29,1,NaN
2,c:1001,1901,c:5,2009-08-10,0,47500000.0
3,c:10014,3878,c:23054,2010-09-30,1,NaN
4,c:100265,6106,c:38215,2011-09-06,1,NaN
...,...,...,...,...,...,...
9406,c:9949,9692,c:267149,2013-09-25,0,400000000.0
9407,c:99685,7240,c:161312,2011-08-01,0,4800000.0
9408,c:997,85,c:29,2007-09-01,1,350000000.0
9409,c:99737,6463,c:67724,2011-11-30,1,NaN


In [33]:
acq_by_company["acquired_c_id"].duplicated().sum()

np.int64(0)

In [34]:
# obj_frs acq
obj_frs_acq = obj_frs.merge(
    acq_by_company,
    left_on ="objects_cfpr_id", 
    right_on="acquired_c_id",
    how="left"
)
print(obj_frs_acq.shape)
print(obj_frs_acq.columns)
print(obj_frs_acq[["objects_cfpr_id", "funding_round_id"]].duplicated().sum())

(483539, 49)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c

##### ◼ ipos

In [35]:
ipo.columns

Index(['ipo_id', 'ipos_c_id', 'valuation_amount', 'valuation_currency_code',
       'raised_amount', 'raised_currency_code', 'public_at', 'stock_symbol',
       'is_ipos_public_at_missing', 'stock_normalized',
       'is_ipos_valuation_private', 'valuation_amount_usd',
       'valuation_currency_rate', 'is_ipos_raised_private',
       'ipo_raised_amount_usd', 'ipo_raised_currency_rate'],
      dtype='object')

In [36]:
cols_use = [
    #ipos
    "ipo_id"                , "ipos_c_id"                   , "public_at"               , 
    "valuation_amount_usd"  , "is_ipos_valuation_private"   , "is_ipos_raised_private"  , 
    "ipo_raised_amount_usd" 
]
ipo = ipo[cols_use].copy()
print(ipo.columns)

Index(['ipo_id', 'ipos_c_id', 'public_at', 'valuation_amount_usd',
       'is_ipos_valuation_private', 'is_ipos_raised_private',
       'ipo_raised_amount_usd'],
      dtype='object')


In [37]:
ipo["public_at"] = pd.to_datetime(ipo["public_at"], errors='coerce')
ipo_by_company = (
    ipo
    .groupby("ipos_c_id")
    .agg(# 첫번째 이벤트
            ipo_id                      = ("ipo_id","first"),
            first_public_at             = ("public_at","min"),   
            valuation_amount_usd        = ("valuation_amount_usd","first"),
            ipo_raised_amount_usd       = ("ipo_raised_amount_usd","first"),
            is_ipos_valuation_private   = ("is_ipos_valuation_private","max"),
            is_ipos_raised_private      = ("is_ipos_raised_private","max")
        )
).reset_index()
ipo_by_company

,ipos_c_id,ipo_id,first_public_at,valuation_amount_usd,ipo_raised_amount_usd,is_ipos_valuation_private,is_ipos_raised_private
0,c:100844,1022,NaT,NaN,NaN,1,1
1,c:10106,1110,NaT,NaN,NaN,1,1
2,c:10166,840,NaT,NaN,NaN,1,1
3,c:10222,804,NaT,NaN,NaN,1,1
4,c:10241,513,NaT,NaN,NaN,1,1
...,...,...,...,...,...,...,...
1248,c:966,28,NaT,NaN,NaN,1,1
1249,c:9743,1153,NaT,NaN,NaN,1,1
1250,c:9786,528,NaT,NaN,NaN,1,1
1251,c:988,1146,NaT,NaN,NaN,1,1


In [38]:
ipo_by_company["ipos_c_id"].duplicated().sum()

np.int64(0)

In [39]:
# obj_frs_acq_ipo ipo
obj_frs_acq_ipo = obj_frs_acq.merge(
    ipo_by_company,
    left_on ="objects_cfpr_id", 
    right_on="ipos_c_id",
    how="left"
)
print(obj_frs_acq_ipo.shape)
print(obj_frs_acq_ipo.columns)
print(obj_frs_acq_ipo[["objects_cfpr_id", "funding_round_id"]].duplicated().sum())

(483539, 56)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c

In [40]:
print(obj.shape)
print(obj_frs.shape)
print(obj_frs_acq.shape)
print(obj_frs_acq_ipo.shape)

(462620, 23)
(483539, 43)
(483539, 49)
(483539, 56)


##### ◼ offices

In [41]:
off.columns

Index(['offices_c_id', 'office_id', 'description', 'city', 'state_code',
       'country_code', 'latitude', 'longitude', 'offices_description_fixed',
       'cat_offices_description', 'offices_state_filled'],
      dtype='object')

In [42]:
cols_use = [
    #offices
    "offices_c_id"  , "office_id"  , "city"       , 
    "country_code"  , "latitude"   , "longitude"  , 
    "cat_offices_description" 
]
off = off[cols_use].copy()
print(off.columns)

Index(['offices_c_id', 'office_id', 'city', 'country_code', 'latitude',
       'longitude', 'cat_offices_description'],
      dtype='object')


In [43]:
off_by_company = (
    off
    .groupby("offices_c_id")
    .agg(
            n_offices        = ("office_id","nunique")
        )
).reset_index()
print(off_by_company.shape)
print(off_by_company.duplicated().sum())

(95043, 2)
0


In [44]:
off_by_company["offices_c_id"].duplicated().sum()

np.int64(0)

In [45]:
# obj_frs_acq_ipo off
obj_frs_acq_ipo_off = obj_frs_acq_ipo.merge(
    off_by_company,
    left_on ="objects_cfpr_id", 
    right_on="offices_c_id",
    how="left"
)
print(obj_frs_acq_ipo_off.shape)
print(obj_frs_acq_ipo_off.columns)
print(obj_frs_acq_ipo_off[["objects_cfpr_id", "funding_round_id"]].duplicated().sum())

(483539, 58)
Index(['objects_cfpr_id', 'founded_at', 'closed_at', 'description',
       'country_code', 'obj_city_fixed', 'first_investment_at',
       'last_investment_at', 'investment_rounds', 'invested_companies',
       'first_funding_at', 'last_funding_at', 'funding_rounds',
       'funding_total_usd', 'relationships', 'cat_obj_status',
       'obj_category_filled', 'cat_obj_overview', 'obj_state_filled',
       'is_obj_funding_total_usd_private', 'success_flag', 'size_bin',
       'time_to_last_round_months', 'funding_round_id', 'fr_c_id', 'funded_at',
       'funding_round_type', 'funding_round_code', 'raised_amount_usd',
       'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants',
       'is_first_round', 'is_last_round', 'funded_year', 'funded_quarter',
       'cat_fr_type', 'num_fr_type', 'log_participants',
       'is_fr_raised_private', 'prev_round_date', 'round_tempo_days',
       'round_tempo_months', 'acquired_c_id', 'acquisition_id',
       'acquiring_c

In [46]:
s_master = obj_frs_acq_ipo_off.copy()

In [47]:
s_master[["objects_cfpr_id", "funding_round_id"]].duplicated().sum()   

np.int64(0)

In [48]:
s_master.to_csv("./data/stat/success_master.csv", encoding="utf-8", index=False)
print("="*60)
print("csv 추출 완료!")
print("="*60)

csv 추출 완료!
